In [ ]:


# Step 1: Install Gradio (if not already available)
!pip install gradio --quiet

import gradio as gr

# Global session memory to simulate persistent conversation
user_sessions = {}

# Basic simulated logic for intent understanding and slot filling
def respond_to_input(user_message, context):
    message = user_message.lower()

    if "schedule" in message:
        return "Sure! How long should the meeting be?", {"intent": "schedule_meeting"}
    elif "hour" in message:
        context["duration"] = "1 hour"
        return "Noted. Do you have a specific day or time in mind?", context
    elif "tuesday" in message:
        context["preferred_time"] = "Tuesday afternoon"
        return "Great! I found 2:00 PM and 4:30 PM available. Which one works for you?", context
    else:
        return "Could you please clarify your scheduling preference?", context

# Simulated Calendar logic
def fake_calendar_auth():
    return "authenticated_session"

def check_time_slots(session, duration, preference):
    return ["2:00 PM", "4:30 PM"]

# Chat handler
def smart_scheduler_agent(message, chat_log):
    try:
        session_id = "demo_user"
        if session_id not in user_sessions:
            user_sessions[session_id] = {"context": {}}

        context = user_sessions[session_id].get("context", {})
        reply, updated_context = respond_to_input(message, context)
        user_sessions[session_id]["context"] = updated_context

        # Suggest times if both duration and preferred time are set
        if updated_context.get("duration") and updated_context.get("preferred_time"):
            session = fake_calendar_auth()
            available_times = check_time_slots(session, updated_context["duration"], updated_context["preferred_time"])
            reply += f"\nAvailable time slots: {', '.join(available_times)}"

        chat_log.append((message, reply))
        return "", chat_log

    except Exception as err:
        chat_log.append(("SYSTEM", f"⚠️ Internal Error: {str(err)}"))
        return "", chat_log

# UI via Gradio Blocks
with gr.Blocks() as interface:
    gr.Markdown("## 🗓️ Smart Scheduler AI Assistant")
    chat_window = gr.Chatbot()
    text_input = gr.Textbox(placeholder="Ask me to schedule a meeting...")
    reset_btn = gr.Button("Clear Chat")

    text_input.submit(smart_scheduler_agent, [text_input, chat_window], [text_input, chat_window])
    reset_btn.click(lambda: [], None, chat_window, queue=False)

interface.launch(share=True)


<ipython-input-7-1516333952>:60: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chat_window = gr.Chatbot()


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d2ef229c9fb2bdf581.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
